In [41]:
import pandas as pd
from dotenv import load_dotenv

import os
import json
import requests
from datetime import datetime
import warnings


pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
warnings.filterwarnings('ignore')


In [7]:
# Load environment variables from .env file
load_dotenv()

True

In [55]:
BUILDKITE_API_TOKEN = os.getenv('BUILDKITE_API_TOKEN')
ORGANIZATION_SLUG='vllm'
PIPELINE_SLUG = 'ci-aws'
TODAY = datetime.utcnow().strftime('%Y-%m-%dT00:00:00Z') # it is UTC, so -2 hours from Finnish local time
WAITING_TIME_ALERT_THR = 14400 # 4 hours


In [9]:
# Set the URL
url = f"https://api.buildkite.com/v2/organizations/{ORGANIZATION_SLUG}/pipelines/{PIPELINE_SLUG}/builds"

# Set the query parameters
params = {
    'created_from': TODAY,
}

# Set the headers
headers = {
    'Authorization': f'Bearer {BUILDKITE_API_TOKEN}'
}

# Make the GET request
response = requests.get(url, headers=headers, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    df = pd.json_normalize(data)
else:
    print(f"Request failed with status code {response.status_code}")

In [10]:
df.head()

,id,graphql_id,url,web_url,number,state,cancel_reason,blocked,blocked_state,message,commit,branch,tag,source,creator,created_at,scheduled_at,started_at,finished_at,pull_request,rebuilt_from,jobs,cluster_id,cluster_url,author.username,author.name,author.email,meta_data.buildkite:git:commit,pipeline.id,pipeline.graphql_id,pipeline.url,pipeline.web_url,pipeline.name,pipeline.description,pipeline.slug,pipeline.repository,pipeline.cluster_id,pipeline.pipeline_template_uuid,pipeline.branch_configuration,pipeline.default_branch,pipeline.skip_queued_branch_builds,pipeline.skip_queued_branch_builds_filter,pipeline.cancel_running_branch_builds,pipeline.cancel_running_branch_builds_filter,pipeline.allow_rebuilds,pipeline.provider.id,pipeline.provider.settings.build_branches,pipeline.provider.settings.build_merge_group_checks_requested,pipeline.provider.settings.build_pull_request_base_branch_changed,pipeline.provider.settings.build_pull_request_forks,pipeline.provider.settings.build_pull_request_labels_changed,pipeline.provider.settings.build_pull_request_ready_for_review,pipeline.provider.settings.build_pull_requests,pipeline.provider.settings.build_tags,pipeline.provider.settings.cancel_deleted_branch_builds,pipeline.provider.settings.filter_enabled,pipeline.provider.settings.prefix_pull_request_fork_branch_names,pipeline.provider.settings.publish_blocked_as_pending,pipeline.provider.settings.publish_commit_status_per_step,pipeline.provider.settings.publish_commit_status,pipeline.provider.settings.pull_request_branch_filter_enabled,pipeline.provider.settings.separate_pull_request_statuses,pipeline.provider.settings.skip_builds_for_existing_commits,pipeline.provider.settings.skip_pull_request_builds_for_existing_commits,pipeline.provider.settings.trigger_mode,pipeline.provider.settings.use_step_key_as_commit_status,pipeline.provider.settings.repository,pipeline.provider.settings.pull_request_branch_filter_configuration,pipeline.provider.settings.filter_condition,pipeline.provider.webhook_url,pipeline.builds_url,pipeline.badge_url,pipeline.created_by.id,pipeline.created_by.graphql_id,pipeline.created_by.name,pipeline.created_by.email,pipeline.created_by.avatar_url,pipeline.created_by.created_at,pipeline.created_at,pipeline.archived_at,pipeline.env,pipeline.scheduled_builds_count,pipeline.running_builds_count,pipeline.scheduled_jobs_count,pipeline.running_jobs_count,pipeline.waiting_jobs_count,pipeline.visibility,pipeline.tags,pipeline.emoji,pipeline.color,pipeline.configuration,pipeline.steps,pipeline.cluster_url,creator.id,creator.graphql_id,creator.name,creator.email,creator.avatar_url,creator.created_at,pull_request.id,pull_request.base,pull_request.repository,author,rebuilt_from.id,rebuilt_from.number,rebuilt_from.url
0,0192f6ab-038e-4e77-983b-b36f78e9a955,QnVpbGQtLS0wMTkyZjZhYi0wMzhlLTRlNzctOTgzYi1iMz...,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683,10683,passed,None,True,,[Bugfix][OpenVINO] Fix circular reference #993...,ccb5376a9a88bb6251c4434b79c173151e6f7729,main,None,webhook,NaN,2024-11-04T10:14:15.758Z,2024-11-04T10:14:15.683Z,2024-11-04T10:14:53.296Z,2024-11-04T11:43:51.605Z,NaN,NaN,[{'id': '0192f6ab-03cc-424b-800c-0b85592bf40c'...,None,None,MengqingCao,Mengqing Cao,cmq0113@163.com,commit ccb5376a9a88bb6251c4434b79c173151e6f772...,018fac60-6fa2-4681-9714-0ffcf8e619e9,UGlwZWxpbmUtLS0wMThmYWM2MC02ZmEyLTQ2ODEtOTcxNC...,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws,CI AWS,CI testing of vLLM repo running on AWS,ci-aws,https://github.com/vllm-project/vllm.git,None,None,main,main,True,!main,True,!main,True,github,True,False,False,True,True,False,True,False,False,True,True,False,True,True,False,True,False,True,code,False,vllm-project/vllm,,"(build.pull_request.labels includes ""ready"" &&...",https://webhook.buildkite.com/deliver/3e7ddff5...,https://api.buildkite.com/v2/organizations/vll...,https://badge.buildkite.com/0f7dfe72085fa6fc9e...,018d38

In [11]:
df.shape

(22, 106)

In [12]:
df.state.value_counts()

state
passed      11
failed       7
canceled     4
Name: count, dtype: int64

In [13]:
useful_columns = ['id', 'web_url', 'url', 'number', 'state', 'cancel_reason', 'blocked', 'blocked_state', 'jobs']

In [14]:
d = df[useful_columns]

In [15]:
d.head()

,id,web_url,url,number,state,cancel_reason,blocked,blocked_state,jobs
0,0192f6ab-038e-4e77-983b-b36f78e9a955,https://buildkite.com/vllm/ci-aws/builds/10683,https://api.buildkite.com/v2/organizations/vll...,10683,passed,None,True,,[{'id': '0192f6ab-03cc-424b-800c-0b85592bf40c'...
1,0192f689-d3dd-4ad7-a037-e96c59255e60,https://buildkite.com/vllm/ci-aws/builds/10682,https://api.buildkite.com/v2/organizations/vll...,10682,passed,None,True,,[{'id': '0192f689-d40e-45a0-9325-bd3147b9105f'...
2,0192f662-2569-4023-a559-5446e984a30f,https://buildkite.com/vllm/ci-aws/builds/10681,https://api.buildkite.com/v2/organizations/vll...,10681,failed,None,False,,[{'id': '0192f662-259a-4579-80fc-ca57f3472504'...
3,0192f648-1a14-4a41-b5c5-09c34bef6169,https://buildkite.com/vllm/ci-aws/builds/10680,https://api.buildkite.com/v2/organizations/vll...,10680,passed,None,True,,[{'id': '0192f648-1a48-4b85-b21f-ae91def0f4d0'...
4,0192f638-5712-4b08-801c-ed9e6b89f128,https://buildkite.com/vllm/ci-aws/builds/10679,https://api.buildkite.com/v2/organizations/vll...,10679,passed,None,True,,[{'id': '0192f638-5744-47b7-b500-ac764c8189c9'...


In [16]:
jobs_df = pd.json_normalize(df['jobs'].explode())
jobs_df.head()

,id,graphql_id,type,name,step_key,agent_query_rules,state,build_url,web_url,log_url,raw_log_url,artifacts_url,command,soft_failed,exit_status,artifact_paths,created_at,scheduled_at,runnable_at,started_at,finished_at,expired_at,retried,retried_in_job_id,retries_count,retry_source,retry_type,parallel_group_index,parallel_group_total,matrix,cluster_id,cluster_url,cluster_queue_id,cluster_queue_url,step.id,step.signature,priority.number,agent.id,agent.url,agent.web_url,agent.name,agent.connection_state,agent.ip_address,agent.hostname,agent.user_agent,agent.version,agent.creator,agent.created_at,agent.job,agent.last_job_finished_at,agent.priority,agent.meta_data,agent.cluster_url,agent.cluster_queue_url,label,unblocked_by,unblocked_at,unblockable,unblock_url,agent,retry_source.job_id,retry_source.retry_type
0,0192f6ab-03cc-424b-800c-0b85592bf40c,Sm9iLS0tMDE5MmY2YWItMDNjYy00MjRiLTgwMGMtMGI4NT...,script,bootstrap,bootstrap,[queue=small_cpu_queue],passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,https://api.buildkite.com/v2/organizations/vll...,https://api.buildkite.com/v2/organizations/vll...,https://api.buildkite.com/v2/organizations/vll...,"if [[ -n """" ]]; then VLLM_CI_BRANCH= curl -sSL...",False,0.0,NaN,2024-11-04T10:14:15.737Z,2024-11-04T10:14:15.737Z,2024-11-04T10:14:15.810Z,2024-11-04T10:14:53.296Z,2024-11-04T10:15:03.428Z,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0192f6ab-03b5-4762-bc27-2cdd132c0b41,NaN,0.0,0192f6ab-93dd-4cc5-9c6b-e51e811da177,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/organizations/vllm/unclu...,bk-small-cpu-queue-i-0e3136c019779f5e9-1,disconnected,54.184.82.161,ip-10-0-110-146.us-west-2.compute.internal,buildkite-agent/3.73.1.8506 (linux; amd64),3.73.1,NaN,2024-11-04T10:14:52.637Z,NaN,2024-11-04T10:15:03.439Z,0.0,"[queue=small_cpu_queue, docker=25.0.3, stack=b...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0192f6ab-b82d-49bf-88ab-f0e8a4ccc227,Sm9iLS0tMDE5MmY2YWItYjgyZC00OWJmLTg4YWItZjBlOG...,script,:docker: build image,image-build,[queue=cpu_queue],passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,https://api.buildkite.com/v2/organizations/vll...,https://api.buildkite.com/v2/organizations/vll...,https://api.buildkite.com/v2/organizations/vll...,aws ecr-public get-login-password --region us-...,False,0.0,NaN,2024-11-04T10:15:01.866Z,2024-11-04T10:14:15.737Z,2024-11-04T10:15:03.517Z,2024-11-04T10:15:35.936Z,2024-11-04T10:36:36.578Z,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0192f6ab-b77a-466e-99df-058598a63151,NaN,0.0,0192f6ac-39a3-4201-96a4-438a1af70dd8,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/organizations/vllm/unclu...,bk-cpu-queue-i-090da303a61bda67e-1,disconnected,34.222.117.105,ip-10-0-31-18.us-west-2.compute.internal,buildkite-agent/3.73.1.8506 (linux; amd64),3.73.1,NaN,2024-11-04T10:15:35.075Z,NaN,2024-11-04T10:36:36.587Z,0.0,"[queue=cpu_queue, docker=25.0.3, stack=bk-cpu-...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0192f6ab-b82f-4ef4-a469-3908ddd58bf6,Sm9iLS0tMDE5MmY2YWItYjgyZi00ZWY0LWE0NjktMzkwOG...,script,Documentation Build,None,[queue=small_cpu_queue],passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,https://api.buildkite.com/v2/organizations/vll...,https://api.buildkite.com/v2/organizations/vll...,https://api.buildkite.com/v2/organizations/vll...,,False,0.0,NaN,2024-11-04T10:15:01.866Z,2024-11-04T10:14:15.737Z,2024-11-04T10:36:36.667Z,2024-11-04T10:37:12.163Z,2024-11-04T10:40:18.092Z,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0192f6ab-b77b-4a38-a6c2-6968943c26fe,NaN,0.0,0192f6c0-01b1-4681-8ba4-c2285b73e043,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/organizations/vllm/unclu...,bk-small-cpu-queue-i-08338b91fe8e5a7a0-1,disconnected,35.91.42.218,ip-10-0-189-148.us-west-2.compute.internal,buildkite-agent/3.73

In [17]:
jobs_df.build_url.nunique()

22

In [18]:
jobs_useful_columns = ['id', 'name', 'state', 'build_url', 'web_url', 'soft_failed', 'created_at', 'scheduled_at', 'runnable_at', 'started_at',	'finished_at', 'expired_at', 'retried', 'agent.id', 'agent.name', 'agent.web_url', 'agent.connection_state', 'agent.meta_data']

In [19]:
j = jobs_df[jobs_useful_columns]
j.head()

,id,name,state,build_url,web_url,soft_failed,created_at,scheduled_at,runnable_at,started_at,finished_at,expired_at,retried,agent.id,agent.name,agent.web_url,agent.connection_state,agent.meta_data
0,0192f6ab-03cc-424b-800c-0b85592bf40c,bootstrap,passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,False,2024-11-04T10:14:15.737Z,2024-11-04T10:14:15.737Z,2024-11-04T10:14:15.810Z,2024-11-04T10:14:53.296Z,2024-11-04T10:15:03.428Z,NaN,False,0192f6ab-93dd-4cc5-9c6b-e51e811da177,bk-small-cpu-queue-i-0e3136c019779f5e9-1,https://buildkite.com/organizations/vllm/unclu...,disconnected,"[queue=small_cpu_queue, docker=25.0.3, stack=b..."
1,0192f6ab-b82d-49bf-88ab-f0e8a4ccc227,:docker: build image,passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,False,2024-11-04T10:15:01.866Z,2024-11-04T10:14:15.737Z,2024-11-04T10:15:03.517Z,2024-11-04T10:15:35.936Z,2024-11-04T10:36:36.578Z,NaN,False,0192f6ac-39a3-4201-96a4-438a1af70dd8,bk-cpu-queue-i-090da303a61bda67e-1,https://buildkite.com/organizations/vllm/unclu...,disconnected,"[queue=cpu_queue, docker=25.0.3, stack=bk-cpu-..."
2,0192f6ab-b82f-4ef4-a469-3908ddd58bf6,Documentation Build,passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,False,2024-11-04T10:15:01.866Z,2024-11-04T10:14:15.737Z,2024-11-04T10:36:36.667Z,2024-11-04T10:37:12.163Z,2024-11-04T10:40:18.092Z,NaN,False,0192f6c0-01b1-4681-8ba4-c2285b73e043,bk-small-cpu-queue-i-08338b91fe8e5a7a0-1,https://buildkite.com/organizations/vllm/unclu...,disconnected,"[queue=small_cpu_queue, docker=25.0.3, stack=b..."
3,0192f6ab-b831-4d62-956d-d4e2b173b962,"Async Engine, Inputs, Utils, Worker Test",passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,False,2024-11-04T10:15:01.866Z,2024-11-04T10:14:15.737Z,2024-11-04T10:36:36.667Z,2024-11-04T10:37:27.559Z,2024-11-04T11:03:36.337Z,NaN,False,0192f6c0-3d58-45da-8c51-0d2e3dc2701a,bk-gpu-1-queue-i-03f47f29dd045cbd2-1,https://buildkite.com/organizations/vllm/unclu...,disconnected,"[queue=gpu_1_queue, docker=25.0.3, stack=bk-gp..."
4,0192f6ab-b832-4c73-8bf2-2ac0e09d5ff3,Basic Correctness Test,passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,False,2024-11-04T10:15:01.866Z,2024-11-04T10:14:15.737Z,2024-11-04T10:36:36.667Z,2024-11-04T10:37:28.713Z,2024-11-04T10:47:07.782Z,NaN,False,0192f6c0-41fb-4481-afd0-b34b2002f02f,bk-gpu-1-queue-i-01d0f3441596e4639-1,https://buildkite.com/organizations/vllm/unclu...,disconnected,"[queue=gpu_1_queue, docker=25.0.3, stack=bk-gp..."


In [20]:
j.web_url.values

array(['https://buildkite.com/vllm/ci-aws/builds/10683#0192f6ab-03cc-424b-800c-0b85592bf40c',
       'https://buildkite.com/vllm/ci-aws/builds/10683#0192f6ab-b82d-49bf-88ab-f0e8a4ccc227',
       'https://buildkite.com/vllm/ci-aws/builds/10683#0192f6ab-b82f-4ef4-a469-3908ddd58bf6',
       ...,
       'https://buildkite.com/vllm/ci-aws/builds/10663#0192f48a-e570-4105-8333-cb6aa6e5c8ac',
       'https://buildkite.com/vllm/ci-aws/builds/10663#0192f48a-e571-459f-91ff-602846ce1b9a',
       'https://buildkite.com/vllm/ci-aws/builds/10662#0192f486-f507-4efa-a36d-5acd2cdfadca'],
      dtype=object)

In [21]:
result_df = d.drop(columns=['jobs']).merge(j.reset_index(drop=True), left_on='url', right_on='build_url', suffixes=['_build', '_job'], how='outer')
result_df.shape 

(1900, 26)

In [ ]:
result_df[result_df.name.isna()].shape # 515 jobs have no name or nothing really except for id_job, state_job, these jobs are either blocked or canceled

(515, 26)

In [29]:
result_df[result_df.name.isna()].state_job.value_counts()

state_job
blocked     436
canceled     79
Name: count, dtype: int64

In [30]:
result_df_amd = result_df[(result_df.name.notna()) & (result_df.name.str.contains('AMD'))]
result_df_amd.shape

(231, 26)

In [32]:
result_df_amd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 231 entries, 51 to 1379
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id_build                231 non-null    object 
 1   web_url_build           231 non-null    object 
 2   url                     231 non-null    object 
 3   number                  231 non-null    float64
 4   state_build             231 non-null    object 
 5   cancel_reason           44 non-null     object 
 6   blocked                 231 non-null    object 
 7   blocked_state           231 non-null    object 
 8   id_job                  231 non-null    object 
 9   name                    231 non-null    object 
 10  state_job               231 non-null    object 
 11  build_url               231 non-null    object 
 12  web_url_job             231 non-null    object 
 13  soft_failed             231 non-null    object 
 14  created_at              231 non-null    objec

In [ ]:
result_df_amd['number'] = result_df_amd['number'].astype('int')

In [60]:
def calculate_wait_time(df):
    now_utc = pd.Timestamp.now(tz='UTC')
    df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')
    df['runnable_at'] = pd.to_datetime(df['runnable_at'], errors='coerce')

    # Calculate the difference in seconds
    df['waited_seconds'] = df.apply(
        lambda row: (row['started_at'] - row['runnable_at']).total_seconds() if pd.notna(row['started_at']) and pd.notna(row['runnable_at']) \
              else (now_utc - row['runnable_at']).total_seconds() if pd.notna(row['started_at']) and pd.isna(row['runnable_at']) \
                else None,
        axis=1
    )
    return df





In [ ]:
result_df_amd = calculate_wait_time(result_df_amd)

In [ ]:
def alert(df, wait_time_thr=WAITING_TIME_ALERT_THR):
    alerts = []
    for index, row in result_df_amd.iterrows():
        if row['waited_seconds'] > wait_time_thr:
            alert_message = f"Job {row['name']} from build number {row['number']} waited for {row['waited_seconds']} seconds (more than {wait_time_thr} or {wait_time_thr/3600} hours). More info at {row['web_url_job']}"
            alerts.append(alert_message)
        if (row['state_job'] == 'failed') & (row['soft_failed'] == False): 
            alert_message = f"Job {row['name']} from build number {row['number']} failed. More info at {row['web_url_job']}"
            alerts.append(alert_message)


    return alerts

In [91]:
#result_df_amd[(result_df_amd.state_job=='failed') & ()]

In [92]:

alerts = alert(result_df_amd)
alerts

['Job AMD: :docker: build image from build number 10665 failed. More info at https://buildkite.com/vllm/ci-aws/builds/10665#0192f4d4-235e-4029-9bc9-44777d50d9de',
 'Job AMD: OpenAI-Compatible Tool Use from build number 10674 failed. More info at https://buildkite.com/vllm/ci-aws/builds/10674#0192f541-8765-41dc-98dd-9b7b67c40998',
 'Job AMD: :docker: build image from build number 10676 failed. More info at https://buildkite.com/vllm/ci-aws/builds/10676#0192f5d5-3c49-4505-b104-ee8b01db00d0',
 'Job AMD: :docker: build image from build number 10678 failed. More info at https://buildkite.com/vllm/ci-aws/builds/10678#0192f608-a116-4cc7-9704-346e3fe7ff9e']

In [78]:
result_df_amd[result_df_amd.state_job=='failed']

,id_build,web_url_build,url,number,state_build,cancel_reason,blocked,blocked_state,id_job,name,state_job,build_url,web_url_job,soft_failed,created_at,scheduled_at,runnable_at,started_at,finished_at,expired_at,retried,agent.id,agent.name,agent.web_url,agent.connection_state,agent.meta_data,waited,waited_seconds
53,0192f48a-c0b2-4cd5-b574-807f34a7fdb1,https://buildkite.com/vllm/ci-aws/builds/10663,https://api.buildkite.com/v2/organizations/vll...,10663,passed,None,True,,0192f48a-e55a-4f0e-8212-b0c8832a52f8,AMD: Entrypoints Test,failed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10663...,True,2024-11-04T00:19:56.276Z,2024-11-04T00:19:46.980Z,2024-11-04 00:39:53.674000+00:00,2024-11-04 00:40:00.420000+00:00,2024-11-04T01:24:01.898Z,NaN,False,0192b565-f0f7-43d1-8ce1-6f5e2cf6546e,gpud573.jax.cs.cpe.ice.amd.com-1,https://buildkite.com/organizations/vllm/unclu...,connected,"[queue=amd, docker=true, mi250=true]",6.746,6.746
58,0192f48a-c0b2-4cd5-b574-807f34a7fdb1,https://buildkite.com/vllm/ci-aws/builds/10663,https://api.buildkite.com/v2/organizations/vll...,10663,passed,None,True,,0192f48a-e563-498a-8675-9e88d887981f,AMD: Kernels Test %N,failed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10663...,True,2024-11-04T00:19:56.276Z,2024-11-04T00:19:46.980Z,2024-11-04 00:39:53.674000+00:00,2024-11-04 00:40:03.511000+00:00,2024-11-04T01:31:16.932Z,NaN,False,0192224e-348f-4799-b625-268260226ef9,gpud523.jax.cs.cpe.ice.amd.com-1,https://buildkite.com/organizations/vllm/unclu...,connected,"[queue=amd, docker=true, mi250=true]",9.837,9.837
124,0192f48f-41b1-4e15-a8e1-04d6d8d616c9,https://buildkite.com/vllm/ci-aws/builds/10664,https://api.buildkite.com/v2/organizations/vll...,10664,passed,None,True,,0192f48f-a804-4d3c-8b52-de11312b7307,AMD: Kernels Test %N,failed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10664...,True,2024-11-04T00:25:08.175Z,2024-11-04T00:24:42.300Z,2024-11-04 00:45:21.255000+00:00,2024-11-04 00:54:28.990000+00:00,2024-11-04T01:45:04.282Z,NaN,False,0192df24-0cae-4861-af82-7b8941d35e62,gpud525.jax.cs.cpe.ice.amd.com-1,https://buildkite.com/organizations/vllm/unclu...,connected,"[queue=amd, docker=true, mi250=true]",547.735,547.735
182,0192f4d3-6e3b-46e4-8855-6071f154005b,https://buildkite.com/vllm/ci-aws/builds/10665,https://api.buildkite.com/v2/organizations/vll...,10665,failed,None,False,,0192f4d4-235e-4029-9bc9-44777d50d9de,AMD: :docker: build image,failed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10665...,False,2024-11-04T01:39:56.258Z,2024-11-04T01:39:10.076Z,2024-11-04 01:39:56.743000+00:00,2024-11-04 01:40:01.900000+00:00,2024-11-04T01:57:23.898Z,NaN,False,0190a38f-f28c-47f6-90ef-45f51f2007b6,rocm-framework-build-04-2,https://buildkite.com/organizations/vllm/unclu...,connected,[queue=amd-cpu],5.157,5.157
514,0192f4ff-a67f-4c87-b1aa-7686ddffd407,https://buildkite.com/vllm/ci-aws/builds/10670,https://api.buildkite.com/v2/organizations/vll...,10670,passed,None,True,,0192f500-7edb-4e06-b39d-d27616a28c4f,AMD: Entrypoints Test,failed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10670...,True,2024-11-04T02:28:23.255Z,2024-11-04T02:27:28.043Z,2024-11-04 02:48:16.026000+00:00,2024-11-04 02:56:11.487000+00:00,2024-11-04T03:40:50.831Z,NaN,False,0192b565-f0f7-43d1-8ce1-6f5e2cf6546e,gpud573.jax.cs.cpe.ice.amd.com-1,https://buildkite.com/organizations/vllm/unclu...,connected,"[queue=amd, docker=true, mi250=true]",475.461,475.461
519,0192f4ff-a67f-4c87-b1aa-7686ddffd407,https://buildkite.com/vllm/ci-aws/builds/10670,https://api.buildkite.com/v2/organizations/vll...,10670,passed,None,True,,0192f500-7ee2-4a55-b804-e23db6a6239a,AMD: Kernels Test %N,failed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10670...,True,2024-11-04T02:28:23.255Z,2024-11-04T02:27:28.043Z,2

In [49]:
#result_df_amd[result_df_amd.runnable_at.isna()][['number', 'name', 'scheduled_at','runnable_at', 'started_at', 'waited']] # doesn't seem to have nan values in only one of runnable_at or started_at


In [54]:
result_df_amd[result_df_amd.number==10683]

,id_build,web_url_build,url,number,state_build,cancel_reason,blocked,blocked_state,id_job,name,state_job,build_url,web_url_job,soft_failed,created_at,scheduled_at,runnable_at,started_at,finished_at,expired_at,retried,agent.id,agent.name,agent.web_url,agent.connection_state,agent.meta_data,waited
1369,0192f6ab-038e-4e77-983b-b36f78e9a955,https://buildkite.com/vllm/ci-aws/builds/10683,https://api.buildkite.com/v2/organizations/vll...,10683,passed,None,True,,0192f6ab-b898-49c6-a68c-cb05f35e8d1c,AMD: :docker: build image,passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,False,2024-11-04T10:15:01.866Z,2024-11-04T10:14:15.737Z,2024-11-04 10:15:02.310000+00:00,2024-11-04 10:15:04.497000+00:00,2024-11-04T10:35:03.712Z,NaN,False,0190a38f-f28c-47f6-90ef-45f51f2007b6,rocm-framework-build-04-2,https://buildkite.com/organizations/vllm/unclu...,connected,[queue=amd-cpu],2.187
1370,0192f6ab-038e-4e77-983b-b36f78e9a955,https://buildkite.com/vllm/ci-aws/builds/10683,https://api.buildkite.com/v2/organizations/vll...,10683,passed,None,True,,0192f6ab-b899-4a2b-bf86-0cf5c43f08af,AMD: Core Test,passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,False,2024-11-04T10:15:01.866Z,2024-11-04T10:14:15.737Z,2024-11-04 10:35:03.809000+00:00,2024-11-04 10:35:09.641000+00:00,2024-11-04T10:49:05.196Z,NaN,False,019222e8-aa03-4c67-90a5-2ae2355c5d4e,gpu1100.jax.cs.cpe.ice.amd.com-1,https://buildkite.com/organizations/vllm/unclu...,connected,"[queue=amd, docker=true, mi250=true]",5.832
1371,0192f6ab-038e-4e77-983b-b36f78e9a955,https://buildkite.com/vllm/ci-aws/builds/10683,https://api.buildkite.com/v2/organizations/vll...,10683,passed,None,True,,0192f6ab-b89b-42b4-8eca-e78a59767103,AMD: Entrypoints Test,failed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,True,2024-11-04T10:15:01.866Z,2024-11-04T10:14:15.737Z,2024-11-04 10:35:03.809000+00:00,2024-11-04 10:35:10.904000+00:00,2024-11-04T11:19:38.290Z,NaN,False,0192df24-0cae-4861-af82-7b8941d35e62,gpud525.jax.cs.cpe.ice.amd.com-1,https://buildkite.com/organizations/vllm/unclu...,connected,"[queue=amd, docker=true, mi250=true]",7.095
1372,0192f6ab-038e-4e77-983b-b36f78e9a955,https://buildkite.com/vllm/ci-aws/builds/10683,https://api.buildkite.com/v2/organizations/vll...,10683,passed,None,True,,0192f6ab-b89c-468a-b74d-87c44e6e22ae,AMD: Regression Test,passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,False,2024-11-04T10:15:01.866Z,2024-11-04T10:14:15.737Z,2024-11-04 10:35:03.809000+00:00,2024-11-04 10:35:11.211000+00:00,2024-11-04T10:40:38.906Z,NaN,False,0192e85e-63f9-49d3-bb43-52e9f61b361f,gpu10eb.jax.cs.cpe.ice.amd.com-1,https://buildkite.com/organizations/vllm/unclu...,connected,"[queue=amd, docker=true, mi250=true]",7.402
1373,0192f6ab-038e-4e77-983b-b36f78e9a955,https://buildkite.com/vllm/ci-aws/builds/10683,https://api.buildkite.com/v2/organizations/vll...,10683,passed,None,True,,0192f6ab-b89e-4fd2-8c94-45f2b08e9190,AMD: Engine Test,passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,False,2024-11-04T10:15:01.866Z,2024-11-04T10:14:15.737Z,2024-11-04 10:35:03.809000+00:00,2024-11-04 10:35:10.823000+00:00,2024-11-04T10:49:22.702Z,NaN,False,01922256-33b6-47ce-bf67-35d0e1bbd089,gpu6931.jax.cs.cpe.ice.amd.com-1,https://buildkite.com/organizations/vllm/unclu...,connected,"[queue=amd, docker=true, mi250=true]",7.014
1374,0192f6ab-038e-4e77-983b-b36f78e9a955,https://buildkite.com/vllm/ci-aws/builds/10683,https://api.buildkite.com/v2/organizations/vll...,10683,passed,None,True,,0192f6ab-b89f-4704-8f8d-1e7b973e96f6,AMD: LogitsProcessor Test,passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,False,2024-11-04T10:15:01.866Z,2024-11-04T10:14:15.737Z,2024-11-04 10:35:03.809000+00:00,2024-11-04 10:35

In [56]:
result_df_amd['agent.name'].value_counts()

agent.name
gpu9496.jax.cs.cpe.ice.amd.com-1    21
gpu1100.jax.cs.cpe.ice.amd.com-1    21
gpud525.jax.cs.cpe.ice.amd.com-1    17
gpu10eb.jax.cs.cpe.ice.amd.com-1    17
gpud523.jax.cs.cpe.ice.amd.com-1    17
gpu6931.jax.cs.cpe.ice.amd.com-1    16
gpud518.jax.cs.cpe.ice.amd.com-1    14
gpu942a.jax.cs.cpe.ice.amd.com-1    14
gpud573.jax.cs.cpe.ice.amd.com-1    13
rocm-framework-build-04-2           10
rocm-framework-build-04-4            9
rocm-framework-build-04-5            2
Name: count, dtype: int64

In [ ]:
# add if fail and it is not soft fail, then alert as well
# alert should include web_url with the run